# UC7 with Attack Variability

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

This notebook demonstrates the PrimAITE environment with the UC7 network laydown and multiple attack personas. The first attack persona is TAP001 which performs a ransomware attack against the database. The other one is TAP003 which is able to maliciously add ACL rules that block green pattern of life.

The environment switches between these two attacks on a pre-defined schedule which is defined in the schedule.yaml file of the scenario folder.

## Setup and Imports

In [ ]:
!primaite setup

In [ ]:
import yaml
from primaite.session.environment import PrimaiteGymEnv
from primaite import PRIMAITE_PATHS
from prettytable import PrettyTable
from deepdiff.diff import DeepDiff
from primaite.session.environment import PrimaiteGymEnv
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.simulator.system.services.dns.dns_server import DNSServer
from primaite.simulator.system.software import SoftwareHealthState
from primaite.simulator.file_system.file_system_item_abc import FileSystemItemHealthStatus
from primaite.simulator.network.hardware.nodes.network.switch import Switch
from primaite.simulator.system.applications.web_browser import WebBrowser
from primaite.simulator.network.container import Network
from primaite.simulator.system.services.service import ServiceOperatingState
from primaite.simulator.network.hardware.node_operating_state import NodeOperatingState
from primaite.simulator.system.services.database.database_service import DatabaseService
from primaite.simulator.system.applications.database_client import DatabaseClient
from primaite.simulator.network.hardware.nodes.network.firewall import Firewall
from primaite.game.game import PrimaiteGame
from primaite.simulator.sim_container import Simulation
from primaite.config.load import load, _EXAMPLE_CFG
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.simulator.network.hardware.nodes.host.computer import Computer

scenario_path = PRIMAITE_PATHS.user_config_path / "example_config/uc7_multiple_attack_variants"

In [ ]:
env = PrimaiteGymEnv(env_config=scenario_path)

## Schedule

Let's print the schedule so that we can see which attack we can expect on each episode.

On episodes 0-4, the TAP001 agent will be used, and on episodes 5-9, the TAP003 agent will be used. Then, the environment will alternate between the two. Furthermore, the TAP001 agent will alternate between starting at `ST_PROJ-A-PRV-PC-1`, `ST_PROJ-B-PRV-PC-2`, `ST_PROJ-C-PRV-PC-3`.

In [ ]:
with open(scenario_path / "schedule.yaml",'r') as f:
    print(f.read())

## TAP001 attack

Let's first demonstrate the TAP001 attack. We will let the environment run for 30 steps and print out the red agent's actions.


In [ ]:
#utils
def run_green_and_red_pol(num_steps):
    for i in range(num_steps): # perform steps
        env.step(0)

def print_agent_actions_except_do_nothing(agent_name):
    """Get the agent's action history, filter out `do-nothing` actions, print relevant data in a table."""
    table = PrettyTable()
    table.field_names = ["Step", "Action", "Node", "Application", "Target IP", "Response"]
    print(f"Episode: {env.episode_counter}, Actions for '{agent_name}':")
    for item in env.game.agents[agent_name].history:
        if item.action == "do-nothing":
            continue

        node, application, target_ip = "N/A", "N/A", "N/A",

        if item.action.startswith("node-nmap"):
            node = item.parameters['source_node']
            application = "nmap"
            target_ip = str(item.parameters['target_ip_address'])
            target_ip = (target_ip[:25]+'...') if len(target_ip)>25 else target_ip # truncate long string

        elif item.action == "router-acl-add-rule":
            node = item.parameters.get("router_name")
        elif item.action == "node-send-remote-command":
            node = item.parameters.get("node_name")
            target_ip = item.parameters.get("remote_ip")
            application = item.parameters.get("command")
        elif item.action == "node-session-remote-login":
            node = item.parameters.get("node_name")
            target_ip = item.parameters.get("remote_ip")
            application = "user-manager"
        elif item.action.startswith("c2-server"):
            application = "c2-server"
            node = item.parameters.get('node_name')
        elif item.action == "configure-c2-beacon":
            application = "c2-beacon"
            node = item.parameters.get('node_name')

        else:
            if (node_id := item.parameters.get('node_id')) is not None:
                node = env.game.agents[agent_name].action_manager.node_names[node_id]
            if (application_id := item.parameters.get('application_id')) is not None:
                application = env.game.agents[agent_name].action_manager.application_names[node_id][application_id]
            if (application_name := item.parameters.get('application_name')) is not None:
                application = application_name

        table.add_row([item.timestep, item.action, node, application, target_ip, item.response.status])

    print(table)
    print("(Any DONOTHING actions are omitted)")

def finish_episode_and_print_reward():
    while env.game.step_counter < 128:
        env.step(0)
    print(f"Total reward this episode: {env.agent.reward_function.total_reward:2f}")



In [ ]:
run_green_and_red_pol(110)
print_agent_actions_except_do_nothing("attacker")

In [ ]:
st_data_prv_srv_db: Server = env.game.simulation.network.get_node_by_hostname("ST_DATA-PRV-SRV-DB")
st_data_prv_srv_db.file_system.show()

In [ ]:
finish_episode_and_print_reward()

## TAP001 Prevention

The blue agent should be able to prevent the ransomware attack by blocking the red agent's access to the database. Let's run the environment until the observation space shows symptoms of the attack starting.

Because we are in episode index 1, the red agent will use `ST-PROJ-A-PRV-PC-1` to start the attack. On step 25, the red agent installs `RansomwareScript`.

In [ ]:
env.reset()
obs, reward, term, trunc, info = env.step(0)
for i in range(25): # we know that the ransomware install happens at step 25
    old = obs
    obs, reward, term, trunc, info = env.step(0)
    new = obs

diff = DeepDiff(old,new)
print(f"Step {env.game.step_counter}") # it's step 26 now because the step counter is incremented after the step
for d,v in diff.get('values_changed', {}).items():
    print(f"{d}: {v['old_value']} -> {v['new_value']}")


We can see that on HOST0, application index 1 has gone from `operating_status` 0 to 3, meaning there wasn't an application before, but now there is an application in the `INSTALLING` state. The blue agent should be able to detect this and block the red agent's access to the database. Action 43 will block `ST-PROJ-A-PRV-PC-1` from sending POSTGRES traffic to the DB server.

If this were a different episode, it could have been `ST-PROJ-B-PRV-PC-2` or `ST-PROJ-C-PRV-PC-3` that are affected, and a different defensive action would be required.

In [ ]:
env.step(43)
env.step(45)
env.step(47)

In [ ]:
st_intra_prv_rt_cr: Router = env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR")
st_intra_prv_rt_cr.acl.show()

In [ ]:
finish_episode_and_print_reward()

In [ ]:
st_intra_prv_rt_cr.acl.show()

Now TAP001 is unable to locate the database!

In [ ]:
print_agent_actions_except_do_nothing("attacker")

## TAP003 attack

Let's skip until episode 5 and demonstrate the TAP003 attack. We will let the environment run and print out the red agent's actions.

By default, TAP003 will add the following rules:

|Target Router         | Impact |
|----------------------|--------|
|`ST_INTRA-PRV-RT-DR-1`| Blocks all `POSTGRES_SERVER` that arrives at the `ST_INTRA-PRV-RT-DR-1` router. This rule will prevent all ST_PROJ_* hosts from accessing the database (`ST_DATA-PRV-SRV-DB`).|
|`ST_INTRA-PRV-RT-CR`| Blocks all `HTTP` traffic that arrives at the`ST_INTRA-PRV-RT-CR` router. This rule will prevent all SOME_TECH hosts from accessing the webserver (`ST-DMZ-PUB-SRV-WEB`)|
|`REM-PUB-RT-DR`| Blocks all `DNS` traffic that arrives at the `REM-PUB-RT-DR` router. This rule prevents any remote site works from accessing the DNS Server (`ISP-PUB-SRV-DNS`).|

In [ ]:
while env.episode_counter < 5:
    env.reset()

In [ ]:
run_green_and_red_pol(128)
print_agent_actions_except_do_nothing("attacker")
obs, reward, term, trunc, info = env.step(0); # one more step so we can capture the value of `obs`

The agent selected to add ACL rules that will prevent green pattern of life by blocking a variety of different traffic. This has a negative impact on reward. Let's view the ACL list on the affected router.

In [ ]:
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1").acl.show()

In [ ]:
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR").acl.show()

In [ ]:
env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR").acl.show()

We can see that at indices 1-5, there are ACL rules that block all traffic. The blue agent can see this rule in the `ROUTERS` part of the observation space.


In [ ]:
obs['NODES']['ROUTER0']['ACL'][1]

In [ ]:
obs['NODES']['ROUTER1']['ACL'][1]

In [ ]:
obs['NODES']['ROUTER2']['ACL'][1]

## Preventing TAP003 attack

The blue agent can prevent the red agent from adding ACL rules. TAP003 relies on connecting to the router via SSH, and sending remote ACL_ADDRULE requests. The blue agent can prevent this by pre-emptively changing the admin password on the affected routers or by blocking SSH traffic between the red agent's starting node and the target routers.

In [ ]:
env.reset()
obs, reward, term, trunc, info = env.step(0)
old = obs
for i in range(128): 
    obs, reward, term, trunc, info = env.step(0)
    new = obs

diff = DeepDiff(old,new)
print(f"Step {env.game.step_counter}") # it's the next step now because the step counter is incremented after the step
for d,v in diff.get('values_changed', {}).items():
    print(f"{d}: {v['old_value']} -> {v['new_value']}")

By printing the reward of each individual agent, we will see what green agents are affected the most. Of course, these green rewards count towards the blue reward so ultimately the blue agent should learn to remove the ACL rule.

In [ ]:
finish_episode_and_print_reward()

for ag in env.game.agents.values():
    print(ag.config.ref, ag.reward_function.total_reward)

The most effective option that the blue agent has against TAP003 is to prevent the red agent from ever adding the ACLs in the first place through blocking the SSH connection.

In [ ]:
env.reset()
env.step(51) # SSH Blocking ACL on ST-INRA-PRV-RT-R1
finish_episode_and_print_reward()

for ag in env.game.agents.values():
    print(ag.config.ref, ag.reward_function.total_reward)

Additionally, another option the blue agent can take is to change the passwords of the different target routers that TAP003 will attack through the `NODE_ACCOUNTS_CHANGE_PASSWORD` action.

In [ ]:
env.reset()
env.step(50) # NODE_ACCOUNTS_CHANGE_PASSWORD | ST_INTRA-prv-rt-cr
env.step(52) # NODE_ACCOUNTS_CHANGE_PASSWORD | ST_INTRA-prv-rt-dr-1
env.step(54) # NODE_ACCOUNTS_CHANGE_PASSWORD | rem-pub-rt-dr
finish_episode_and_print_reward()

for ag in env.game.agents.values():
    print(ag.config.ref, ag.reward_function.total_reward)

Lastly, the blue agent can remedy the impacts of TAP003 through removing the malicious ACLs that TAP003 adds.

In [ ]:
env.reset()

# Allow TAP003 to add it's malicious rules
for _ in range(45):
    env.step(0)

env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR").acl.show()
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1").acl.show()
env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR").acl.show()

In [ ]:
env.step(44) # ROUTER_ACL_REMOVERULE | ST_INTRA-prv-rt-cr
env.step(53) # ROUTER_ACL_REMOVERULE | ST_INTRA-prv-rt-dr-1
env.step(55) # ROUTER_ACL_REMOVERULE | rem-pub-rt-dr

In [ ]:
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-CR").acl.show()
env.game.simulation.network.get_node_by_hostname("ST_INTRA-PRV-RT-DR-1").acl.show()
env.game.simulation.network.get_node_by_hostname("REM-PUB-RT-DR").acl.show()


In [ ]:
finish_episode_and_print_reward()

for ag in env.game.agents.values():
    print(ag.config.ref, ag.reward_function.total_reward)